In [1]:
from urllib import request
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle

In [2]:
driver = webdriver.Chrome("./chromedriver")

driver.get("https://wikiway.com/strani/")
soup = BeautifulSoup(driver.page_source, 'html.parser')

countries={}
pr = [".", "'", "("] #сразу убрать то, что будет сложно вводить
for el in soup.find('div', {'class' : "spisok-stran"}).find_all('ul', {'class' : "country"}):
    for c in el:
        try:
            if not any(_ in c.text.strip() for _ in pr):
                countries[c.text.strip()]=c.a.get('href')
            #print ('country: ', c.text.strip(), c.a.get('href'))
        except AttributeError: #если текста нет
            pass
        
driver.quit()

In [3]:
def unpack(data):
    input = open(data, 'rb')
    obj = pickle.load(input)
    input.close()
    return obj


In [ ]:
count_unigrams = unpack('1stemgrams.data')
count_bigrams = unpack('2grams.data')
count_trigrams = unpack('ngrams.data')

In [12]:
freq_d = {}
for k in countries.keys():
    if ' ' not in k:
        freq = count_unigrams.get(' '+k.lower(), 0)
        #print(1, k, freq)
    elif k.count(' ')==1:
        freq = count_bigrams.get(' '+k.lower(), 0)
        #print(2, k, freq)
    else:
        freq = count_trigrams.get(' '+k.lower(), 0)
        #print(3, k, freq)
    if freq>2000:
        freq_d[k]=freq
#print(len(freq_d))
#print(freq_d.keys())

new_countries = dict((k,countries[k]) for k in countries.keys() if k in freq_d)
#print(len(new_countries))

29


In [47]:
info={}
flag = {}
for c in new_countries:
    driver = webdriver.Chrome("./chromedriver")
    driver.get("https://wikiway.com/"+new_countries[c][1:])
    soup2 = BeautifulSoup(driver.page_source, 'html.parser')
    '''for el in soup2.find('div', {'class' : "info-el"}).find('a'):
        #print('1', el)
        try:
            #print(el.get('src'))
            flag[c]='https://wikiway.com'+el.get('src')
            #print ('country: ', c, p.text.strip())
        except AttributeError: 
            pass'''
    
    p = soup2.find('div', {'class' : "contents"}).find('p')
    #print(p.text.strip())
    info[c]=p.text.strip()
    driver.quit()

In [39]:
print(len(flag))
print(len(info))

29
29


In [40]:
from urllib.request import *

images={}
path='C:/Users/Aleksandra/Documents/renpy-7.1.1-sdk/Knowledge_Island/game/images/'
for f in flag.keys():
    new_url=path+f+'.png'
    images[f]=new_url
    urlretrieve(flag[f], new_url)

In [48]:
final = dict((k,(images[k],info[k])) for k in images.keys())

In [49]:
final

{'Австралия': ('C:/Users/Aleksandra/Documents/renpy-7.1.1-sdk/Knowledge_Island/game/images/Австралия.png',
  'Австралия — государство в Южном полушарии, расположенное на одноимённом материке, а также близлежащих островах Тихого и Индийского океанов. При описании Австралии люди часто говорят об уникальности того или иного факта. Даже само расположение этой страны уникально — Австралия единственное в мире государство, занимающее территорию всего материка. Самый крупный остров Тасмания лежит к югу от континента. Официальное название страны — Австралийский союз. Животный мир и растительный мир Австралии настолько удивителен, что оставит у вас незабываемые впечатления!'),
 'Австрия': ('C:/Users/Aleksandra/Documents/renpy-7.1.1-sdk/Knowledge_Island/game/images/Австрия.png',
  'Австрия не нуждается в представлении. Ее великолепные альпийские пейзажи европейцы полюбили еще в XVIII в., когда и зародилась туристическая индустрия. Но жизнь, скрывающаяся за туристическими плакатами, гораздо интере

In [53]:
def pack(data,obj):
    output = open(data, 'wb')
    pickle.dump(obj, output)
    output.close()

In [54]:
pack('countries.data', final)

In [4]:
final = unpack('countries.data')

In [7]:
final1 = {f:final[f][1] for f in final}
final1

{'Австралия': 'Австралия — государство в Южном полушарии, расположенное на одноимённом материке, а также близлежащих островах Тихого и Индийского океанов. При описании Австралии люди часто говорят об уникальности того или иного факта. Даже само расположение этой страны уникально — Австралия единственное в мире государство, занимающее территорию всего материка. Самый крупный остров Тасмания лежит к югу от континента. Официальное название страны — Австралийский союз. Животный мир и растительный мир Австралии настолько удивителен, что оставит у вас незабываемые впечатления!',
 'Австрия': 'Австрия не нуждается в представлении. Ее великолепные альпийские пейзажи европейцы полюбили еще в XVIII в., когда и зародилась туристическая индустрия. Но жизнь, скрывающаяся за туристическими плакатами, гораздо интереснее, а сегодня интересна, как никогда. После того как в мае 2004 г. в Европейский союз (ЕС) вошли еще десять стран, Австрия оказалась в самом сердце Европы. Вена, некогда столица огромной 

In [25]:
stroka = ''
for f in final1.keys():
    stroka+=f+'\t'+final1[f]+'\n'
stroka

"Австралия\tАвстралия — государство в Южном полушарии, расположенное на одноимённом материке, а также близлежащих островах Тихого и Индийского океанов. При описании Австралии люди часто говорят об уникальности того или иного факта. Даже само расположение этой страны уникально — Австралия единственное в мире государство, занимающее территорию всего материка. Самый крупный остров Тасмания лежит к югу от континента. Официальное название страны — Австралийский союз. Животный мир и растительный мир Австралии настолько удивителен, что оставит у вас незабываемые впечатления!\nАвстрия\tАвстрия не нуждается в представлении. Ее великолепные альпийские пейзажи европейцы полюбили еще в XVIII в., когда и зародилась туристическая индустрия. Но жизнь, скрывающаяся за туристическими плакатами, гораздо интереснее, а сегодня интересна, как никогда. После того как в мае 2004 г. в Европейский союз (ЕС) вошли еще десять стран, Австрия оказалась в самом сердце Европы. Вена, некогда столица огромной многонац

In [51]:
with open("countries.txt", "w", encoding="utf-8") as f:
    f.write(stroka)

In [19]:
new_s = []
for el in stroka[:].split('\n'):
    new_s.append(tuple(el.split('\t')))

In [20]:
new_s

[('Австралия',
  'Австралия — государство в Южном полушарии, расположенное на одноимённом материке, а также близлежащих островах Тихого и Индийского океанов. При описании Австралии люди часто говорят об уникальности того или иного факта. Даже само расположение этой страны уникально — Австралия единственное в мире государство, занимающее территорию всего материка. Самый крупный остров Тасмания лежит к югу от континента. Официальное название страны — Австралийский союз. Животный мир и растительный мир Австралии настолько удивителен, что оставит у вас незабываемые впечатления!'),
 ('Австрия',
  'Австрия не нуждается в представлении. Ее великолепные альпийские пейзажи европейцы полюбили еще в XVIII в., когда и зародилась туристическая индустрия. Но жизнь, скрывающаяся за туристическими плакатами, гораздо интереснее, а сегодня интересна, как никогда. После того как в мае 2004 г. в Европейский союз (ЕС) вошли еще десять стран, Австрия оказалась в самом сердце Европы. Вена, некогда столица ог

In [31]:
with open("countries.txt", "r", encoding="utf-8") as f:
    st = f.read()

In [33]:
new_s = []
for el in st.split('\n'):
    new_s.append(tuple(el.split('\t')))
#final = {el[0]:el[1] for el in new_s}

#countries_to_use = list(final.keys())[:10]

In [48]:
print(new_s[:-1])

[('Австралия', 'Австралия — государство в Южном полушарии, расположенное на одноимённом материке, а также близлежащих островах Тихого и Индийского океанов. При описании Австралии люди часто говорят об уникальности того или иного факта. Даже само расположение этой страны уникально — Австралия единственное в мире государство, занимающее территорию всего материка. Самый крупный остров Тасмания лежит к югу от континента. Официальное название страны — Австралийский союз. Животный мир и растительный мир Австралии настолько удивителен, что оставит у вас незабываемые впечатления!'), ('Австрия', 'Австрия не нуждается в представлении. Ее великолепные альпийские пейзажи европейцы полюбили еще в XVIII в., когда и зародилась туристическая индустрия. Но жизнь, скрывающаяся за туристическими плакатами, гораздо интереснее, а сегодня интересна, как никогда. После того как в мае 2004 г. в Европейский союз (ЕС) вошли еще десять стран, Австрия оказалась в самом сердце Европы. Вена, некогда столица огромно

In [45]:
final = {el[0]:el[1] for el in new_s[:-1]}

countries_to_use = list(final.keys())[:10]
countries_to_use

['Австралия',
 'Австрия',
 'Афганистан',
 'Белоруссия',
 'Болгария',
 'Великобритания',
 'Германия',
 'Греция',
 'Грузия',
 'Египет']

In [47]:
countries_to_use[9]

'Египет'

In [ ]:
i = 0
life = 3

while i<10:
    country_new = countries_to_use[i]
    country_info = final[country_new][1]

    country = str(renpy.input("Угадай стрррррану:"))
    if country == country_new:
        if i < 9:
            cat_geo "Молодец! Вот тебе немного информации:"
        else:
            cat_geo "Уррррраа! Ты прошел уровень!"

            $ score.append('level2')
            if len(score)==3:
                cat_geo "Молодец! Я дарю тебе мачту для твоего корабля!"
